## Import

In [ ]:
## import 
import pandas as pd
import numpy as np
from supervised.automl import AutoML

# vis
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
%matplotlib inline
import seaborn as sns

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

from category_encoders.target_encoder import TargetEncoder

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")

# import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# 시드값 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(2023)

# 데이터프레임 컬럼 전체 보기
pd.set_option('display.max_columns', None)

## 데이터 로드

In [ ]:
path = 'path to your data'

train_org = pd.read(path+'train.csv')
test_org = pd.read_csv(path+'test.csv')

## 전처리 (기본)

→ 날짜, 도시/구/동 , 공휴일, 계절, 시간 코사인 변환

In [ ]:
for df in [train_org, test_org]:
    # 날짜 데이터
    time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

    df[['연', '월', '일', '시간']] = df['사고일시'].str.extract(time_pattern)
    df[['연', '월', '일', '시간']] = df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    # df= df.drop(columns=['사고일시']) # 일단 냄겨두자

    # 도시/구/동 분리
    location_pattern = r'(\S+) (\S+) (\S+)'

    df[['도시', '구', '동']] = df['시군구'].str.extract(location_pattern)
    # df= df.drop(columns=['시군구']) # 얘도 일단 냄겨놔

    # 시간 >> 코사인 변환
    df['cos_hour'] = np.cos(2 * np.pi * df['시간'] / 24)

    # 공휴일 (주말과 공휴일을 묶음)
    # 공휴일 = 1, 비공휴일 = 0
    holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']
    df['dow'] = df['사고일시'].dt.dayofweek #임시로 만든 컬럼
    df['holiday'] = np.where((df['dow']>=5) | (df['사고일시'].dt.strftime('%Y-%m-%d').isin(holi_weekday)),1,0)
    df.drop('dow',axis=1,inplace=True)

### 불필요 칼럼 드롭

In [ ]:
drop_columns = ['ID','도시'] # 어차피 다 대구광역시
train_df = train_org.drop(columns = drop_columns)
test_df = test_org.drop(columns = drop_columns)

## 외부 데이터 merge (결측치 보간된 데이터)

In [ ]:
# load data
light_df = pd.read_csv(path+'동별_보안등개수.csv')
parking_df = pd.read_csv(path+'주차장급지구분별개수.csv')
speed_df = pd.read_csv(path+'동별_제한속도.csv')
cross_walk_df = pd.read_csv(path+'동별_횡단보도개수.csv')
child_area_df = pd.read_csv(path+'동별_어린이보호구역개수.csv')


# merge data
train_df = pd.merge(train_df, light_df, how='left', on=['구','동'])
train_df = pd.merge(train_df, parking_df, how='left', on=['구','동'])
train_df = pd.merge(train_df, speed_df, how='left', on='동')
train_df = pd.merge(train_df, cross_walk_df, how='left', on='동')
train_df = pd.merge(train_df, child_area_df, how='left', on=['구','동'])

test_df = pd.merge(test_df, light_df, how='left', on=['구','동'])
test_df = pd.merge(test_df, parking_df, how='left', on=['구','동'])
test_df = pd.merge(test_df, speed_df, how='left', on='동')
test_df = pd.merge(test_df, cross_walk_df, how='left', on='동')
test_df = pd.merge(test_df, child_area_df, how='left', on=['구','동'])


## 이상치 및 결측값 처리

In [ ]:
# 결측값 확인 
train_df.isnull().sum().sort_values(ascending=False)[:11]
test_df.isnull().sum().sort_values(ascending=False)[:11]

# 일단 interpolate
for df in [train_df,test_df]:
    df['어린이보호구역개수'] = df['어린이보호구역개수'].interpolate()
    df['급지구분_1'] = df['급지구분_1'].interpolate()
    df['급지구분_2'] = df['급지구분_2'].interpolate()
    df['급지구분_3'] = df['급지구분_3'].interpolate()
    df['제한속도'] = df['제한속도'].interpolate()
    df['횡단보도개수'] = df['횡단보도개수'].interpolate()
    
# 급지구분 interpolate하면 안 채워지는 게 1개 있어서 0으로 대체
test_df['급지구분_1'] = test_df['급지구분_1'].fillna(0)
test_df['급지구분_2'] = test_df['급지구분_2'].fillna(0)
test_df['급지구분_3'] = test_df['급지구분_3'].fillna(0)

## 범주형 데이터 인코딩

#### 원핫인코딩

In [ ]:
# 원핫 : 구, 요일

# train_df와 test_df의 기상상태 및 노면상태 열 선택
train_categorical_data = train_df[['구', '요일']]
test_categorical_data = test_df[['구', '요일']]

# OneHotEncoder 인스턴스 생성 및 fit_transform 수행
encoder = OneHotEncoder()
train_encoded = encoder.fit_transform(train_categorical_data)
test_encoded = encoder.transform(test_categorical_data)

# OneHotEncoder가 사용한 카테고리 목록을 가져와서 카테고리 이름을 열 이름으로 변환
feature_names = encoder.get_feature_names_out(['구'])

# 밀집 행렬로 변환 (선택 사항)
train_encoded_dense = train_encoded.toarray()
test_encoded_dense = test_encoded.toarray()

# 데이터프레임으로 변환 (선택 사항)

train_encoded_df = pd.DataFrame(train_encoded_dense, columns=feature_names, index=train_df.index)
test_encoded_df = pd.DataFrame(test_encoded_dense, columns=feature_names, index=test_df.index)

# 기존 열 제거
train_df = train_df.drop(['구','요일'], axis=1)
test_df = test_df.drop(['구','요일'], axis=1)

# 인코딩된 열 추가
train_df = pd.concat([train_df, train_encoded_df], axis=1)
test_df = pd.concat([test_df, test_encoded_df], axis=1)

#### 타겟 인코딩

In [ ]:
# 타겟 인코딩 : 연, 월, 동
# Target encoding

categorical_features = ['연','월','도로형태','사고유형','동','기상상태', '노면상태','season']

for i in categorical_features:
    tr_encoder = TargetEncoder(cols=[i])
    train_df[i] = tr_encoder.fit_transform(train_df[i], train_df['ECLO'])
    test_df[i] = tr_encoder.transform(test_df[i])

## 데이터 분할 및 모델링

In [ ]:
train_x = train_df[test_df.columns] # test 셋에만 있는 컬럼만 추출
train_y = np.log1p(train_df['ECLO']) # 타겟 로그 변환
test_x = test_df

x_train, x_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.2,
			shuffle=True, random_state=42)

matplotlib.use('Agg')

In [ ]:
automl = AutoML(mode="Compete",
                results_path = './AutoML_ver_4/',
                algorithms = ['LightGBM', 'Xgboost', 'CatBoost'],
                n_jobs = -1,
                total_time_limit=30000, 
                eval_metric="rmse", 
                ml_task = "regression",
                explain_level=1, 
                random_state=123)

automl.fit(train_x, train_y)
pred = automl.predict(test_x)

## 제출

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')
submission['ECLO'] = np.expm1(pred) # 타겟값 지수 변환
submission.loc[ submission['ECLO'] < 0.0, 'ECLO'] = 0.0
submission

In [ ]:
submit_path = "../data/submits/automl_1206_001.csv"
submission.to_csv(submit_path, index=False)